# Caliban Fig8 Upload Pipeline
This pipeline creates a Figure Eight job and uploads data files to an S3 bucket for data curation.

- Resize npz into smaller pieces (image dimensions) if needed
- Shorten length of npz if needed
- Upload files to AWS
- Create job csv and upload it to figure 8

**Note: if you need to start a Caliban job to correct the results of a previous Caliban job, please set "base_dir" then skip to the end of the notebook.**

In [1]:
# import statements
from __future__ import absolute_import


from imageio import imread, volread, imwrite, volwrite
import numpy as np
import os
import stat
import sys


import skimage.io as io

from caliban_toolbox.pre_annotation import npz_preprocessing
from caliban_toolbox.post_annotation import npz_postprocessing
from caliban_toolbox.pre_annotation.aws_upload import aws_caliban_upload
from caliban_toolbox.pre_annotation.caliban_csv import initial_csv_maker, create_next_CSV
from caliban_toolbox.pre_annotation.fig_eight_upload import fig_eight



from ipywidgets import fixed, interactive
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

from skimage import filters, img_as_uint
import skimage as sk

from caliban_toolbox.utils.io_utils import get_img_names
from caliban_toolbox.utils import widget_utils

perm_mod = stat.S_IRWXO | stat.S_IRWXU | stat.S_IRWXG

## Load data for model training
We'll specify which channels will be used to generate preliminary labels for the model


In [2]:
data_dir = '/example_data/3D/FOV_0'
data_vol = os.path.join(data_dir, 'Pos0_DAPIRegistered.tif')
data_stack = volread(data_vol)
data_stack = np.expand_dims(data_stack, axis=-1)
data_stack.shape

(38, 1024, 1024, 1)

## Run the data through the network to produce labels

In [ ]:
# deepcell upload code here

## Postprocess the deepcell labels

In [2]:
# code to postprocess labels, select appropriate parameters

# Part A: Adjust image contrast, background, thresholding

In [9]:
# initialize lists to hold newly created channels
adjusted_channels, adjusted_channel_names, adjusted_channel_kwargs = [], [], []

### Step 1: Pick raw image
This will be used as an example to display effect of adjustments later on. This can be changed at any time!

In [3]:
choose_img_output = interactive(widget_utils.choose_img_from_stack, stack = fixed(data_stack), 
                         slice_index = (0, data_stack.shape[0]-1, 1),
                         chan_index = (0, data_stack.shape[3]-1, 1));
choose_img_output

interactive(children=(IntSlider(value=18, description='slice_index', max=37), IntSlider(value=0, description='…

### Step 2: Set raw image adjust parameters
These will be held in memory until npz is saved. A record of the contrast adjustment settings will be saved with them.

In [4]:
# get most recent parameters for selected image
selected_slice_idx, selected_chan_idx = choose_img_output.result
img = data_stack[selected_slice_idx, :, :, selected_chan_idx]

# interative edit mode
adjust_image_output = interactive(widget_utils.adjust_image_interactive, image=fixed(img), blur=(0.0,4,0.1), 
                       gamma_adjust=(0.1,4,0.1), sobel_factor=(10,10000,100), v_min = (0, 255, 1), 
                       v_max = (0, 255, 1));
adjust_image_output

interactive(children=(FloatSlider(value=1.0, description='blur', max=4.0), Checkbox(value=True, description='s…

### Step 3: Adjust raw image with specified parameters

In [6]:
# create placeholder channel to hold the output of channel adjustment
adjusted_channel = np.zeros(data_stack.shape[:-1] + (1,), np.uint8)

# adjust all slices for given channel
for i in range(data_stack.shape[0]):
    image = data_stack[i, :, :, selected_chan_idx]
    adjusted_channel[i, :, :, selected_chan_idx] = widget_utils.adjust_image(image, adjust_image_output.kwargs)

### Step 4: Verify that adjustment looks good across slices

In [7]:
check_adjustment_output = interactive(widget_utils.choose_img_from_stack, stack = fixed(adjusted_channel), 
                               slice_index = (0, data_stack.shape[0]-1, 1),
                              chan_index = fixed(selected_chan_idx));
check_adjustment_output

interactive(children=(IntSlider(value=18, description='slice_index', max=37), Output()), _dom_classes=('widget…

### Step 5: Give the adjusted channel an informative name, repeat part A for each channel that needs to be adjusted

In [11]:
adjusted_channel_name = ["phase_contrast_adjusted"]
adjusted_channel_names.append(adjusted_channel_name)
adjusted_channels.append(adjusted_channel)
adjusted_channel_kwargs.append(adjust_image_output.kwargs)

# Part B: create overlays

In [17]:
# initialize lists to hold newly created channels
combined_channels, combined_channel_names, combined_channel_kwargs = [], [], []

### Step 1: Select first channel to be included in overlay, perform any needed adjustments

In [12]:
img_1_idx = 0
img1 = data_stack[selected_slice_idx, :, :, img_1_idx]

adjust_overlay_1_output = interactive(widget_utils.adjust_image_interactive, image=fixed(img1), blur=(0.0,4,0.1), 
                          gamma_adjust=(0.1,4,0.1), sobel_factor=(10,10000,100), v_min = (0, 255, 1), 
                          v_max = (0, 255, 1));
adjust_overlay_1_output

interactive(children=(FloatSlider(value=1.0, description='blur', max=4.0), Checkbox(value=True, description='s…

### Step 2: Select second channel to be included in overlay, perform any needed adjustments

In [13]:
img_2_idx = 0
img2 = data_stack[selected_slice_idx, :, :, img_2_idx]

adjust_overlay_2_output = interactive(widget_utils.adjust_image_interactive, image=fixed(img2), blur=(0.0,4,0.1), 
                          gamma_adjust=(0.1,4,0.1), sobel_factor=(2,10000,100), v_min = (0, 255, 1), 
                          v_max = (0, 255, 1));
adjust_overlay_2_output

interactive(children=(FloatSlider(value=1.0, description='blur', max=4.0), Checkbox(value=True, description='s…

### Step 3: Select the appropriate settings for combinging the two images together

In [14]:
overlay_images_output = interactive(widget_utils.overlay_images_interactive, 
                               img_1 = fixed(adjust_overlay_1_output.result), 
                               img_2 = fixed(adjust_overlay_2_output.result), 
                               prop_img_1 =(0,1.0, 0.1), v_min = (0, 255, 1), v_max = (0, 255, 1))
overlay_images_output

interactive(children=(FloatSlider(value=0.5, description='prop_img_1', max=1.0), IntSlider(value=0, descriptio…

### Step 4: Create overlays across the whole image stack

In [15]:
combined_settings = overlay_images_output.kwargs
prop_img_1 = combined_settings['prop_img_1']
v_min = combined_settings['v_min']
v_max = combined_settings['v_max']

overlay_channel = np.zeros(data_stack.shape[:-1] + (1,), np.uint8)
for i in range(data_stack.shape[0]):
    image1 = data_stack[i, :, :, img_1_idx]
    image2 = data_stack[i, :, :, img_2_idx]

    image1_adjusted = widget_utils.adjust_image(image1, adjust_overlay_1_output.kwargs)
    image2_adjusted = widget_utils.adjust_image(image2, adjust_overlay_2_output.kwargs)
    overlay_channel[i, :, :, 0] = widget_utils.overlay_images(image1_adjusted, image1_adjusted, 
                                                              prop_img_1, v_min, v_max)

In [16]:
check_adjustment = interactive(widget_utils.choose_img_from_stack, stack = fixed(overlay_channel), 
                         slice_index = (0, data_stack.shape[0]-1, 1),
                         chan_index = fixed(0));
check_adjustment

interactive(children=(IntSlider(value=18, description='slice_index', max=37), Output()), _dom_classes=('widget…

### Step 5: Give the combined channel an informative name, repeat part B for each set of overlays that needs to be constructed

In [18]:
combined_channel_name = ["phase_contrast_adjusted"]
combined_channel_names.append(adjusted_channel_name)
combined_channels.append(overlay_channel)
adjusted_channel_kwargs.append([prop_img_1, v_min, v_max, adjust_overlay_1_output.kwargs, adjust_overlay_2_output])

## Split npz into pieces
The idea is that you'd be starting from one huge npz and breaking it into managable pieces. This part will probably include:
- reshape npz (size of each piece is smaller, but same number of frames)
- break up each npz into fewer frames (annotator does not necessarily need to do all 30+ frames of a movie at once)
- save these reshaped pieces as individual npz files, so they can be uploaded and worked on separately
- relabel the npzs as needed (choose between no relabel, relabel each cell in each frame to have unique label, or relabel to have unique labels but preserve 3D relationships)

These pieces will need specific names so that we can put them back together again if needed (especially putting frames back into longer contiguous movies).

In [2]:
base_dir = '../example_data/'

### Reshape npz (y and x dimensions) if needed

In [ ]:
full_npz_path = '/data/figure_eight/HeLa-S3_cyto_movies/set7/HeLa_movie_s7_uncorrected_fullsize_all_channels.npz'
x_size = 320
y_size = 270
reshaped_save_dir = os.path.join(base_dir, 'reshaped')

In [ ]:
reshape_npz(full_npz_path, x_size, y_size, save_dir = reshaped_save_dir)

### Crop npz into smaller x and y pieces
optionally takes an npz file and splits it into many smaller npzs, each of which can be submitted a separate Figure8 job

### Slice (t or z dimension) single npz if needed
#### Use this option if you do not need to reshape y and x in your npz before beginning work.
This option may be useful in the future for something like curation of 2D npzs. In that case, each slice is independent, so only one slice would be launched at a time. With only one frame to annotate, the annotation time per image allows for a larger image (annotating a larger image would also reduce the number of annotations on the edge, which can be tougher since there is less context to judge cell boundaries).

However, this option has limited use until Caliban supports zooming in and out.

In [ ]:
# full_npz_path = ''
# batch_size = 1
# sliced_save_dir = ''

In [ ]:
# slice_npz_batches(full_npz_path, batch_size, save_dir)

### Slice (t or z dimension) folder of reshaped npzs if needed

In [ ]:
batch_size = 5
sliced_save_dir = os.path.join(base_dir, 'reshaped_resized')

In [ ]:
slice_npz_folder(src_folder = reshaped_save_dir, batch_size = batch_size, save_dir = sliced_save_dir)

### Relabel npzs -- recommended for fig8 first pass jobs
"Predict" relabeling is recommended (unless 3D segmentation models are being used), since this relabeling strategy will perform decently on most 3D data to reduce the human labor involved in correction.

In [ ]:
relabel_npzs_folder(npz_dir = sliced_save_dir, relabel_type = 'predict')